In [1]:
seed = 0
import torch
torch.manual_seed(seed)
import random
random.seed(seed)
import numpy as np
np.random.seed(seed)
torch.use_deterministic_algorithms(True)

In [2]:
import torch
from torch import Tensor
from torch_geometric.logging import init_wandb, log
from torch_geometric.datasets import Planetoid
import torch.nn.functional as F
from utils import train, test, edgeindex2adj
# citeseer = Planetoid(root='.', name='Citeseer')
cora = Planetoid(root='.', name='Cora')
# pubmed = Planetoid(root='.', name='Pubmed')

In [3]:
from models import ALP, GCN
dataset = cora
data = dataset[0]

# take k random training nodes for each class
k = 2
for c in data.y.unique():
    idx = ((data.y == c) & data.train_mask).nonzero(as_tuple=False).view(-1)
    idx = idx[torch.randperm(idx.size(0))]
    idx = idx[k:]
    data.train_mask[idx] = False

print(data.train_mask.sum())

tensor(14)


In [4]:
yshape = dataset[0].y.shape[0]
model = ALP(num_layers=12, alpha=0.99, yshape=yshape)
outs = model.lpa(dataset)

gcn = GCN(dataset.num_features, 8, dataset.num_classes)

optimizer_gcn = torch.optim.Adam(gcn.parameters(), lr=0.01, weight_decay=5e-4)
optimizer_alp = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

In [5]:
model.test()

[1.0, 0.468, 0.487]

In [6]:
train_mask = data.train_mask.clone()
val_mask = data.val_mask.clone()
test_mask = data.test_mask.clone()

unlab_mask = torch.ones_like(data.train_mask)
unlab_mask[(data.train_mask == True) | (data.val_mask == True) | (data.test_mask == True)] = False

print(train_mask.sum(), val_mask.sum(), test_mask.sum(), unlab_mask.sum())

tensor(14) tensor(500) tensor(1000) tensor(1194)


In [7]:
# for p in model.parameters():
#     print(p)

In [8]:
model.train()
best_val_acc = final_test_acc = 0
for epoch in range(1, 200):
    
    optimizer_gcn.zero_grad()
    optimizer_alp.zero_grad()

    # add 1% of unlabelled data to training data
    new_train_mask = train_mask.clone()
    to_add_mask = unlab_mask.clone()
    # put 99% to 0
    idx = to_add_mask.nonzero(as_tuple=False).view(-1)
    idx = idx[torch.randperm(idx.size(0))][:int(0.98 * idx.size(0))]
    to_add_mask[idx] = False
    new_train_mask[to_add_mask] = True
    # print(new_train_mask.sum())
    
    op_alp = model()
    op = gcn(data.x, data.edge_index)
    
    # print(op_alp.shape, op.shape, data.y[train_mask].shape)
    loss = F.cross_entropy(op[new_train_mask, :], op_alp[new_train_mask].argmax(dim=1))
    # print(op_alp[train_mask].shape, data.y[train_mask].shape)
    loss2 = F.cross_entropy(op_alp[train_mask], data.y[train_mask])
    # print(loss.item(), loss2.item())
    # for p in model.parameters():
    #     print(p)
    loss += 0.01 * loss2
    loss.backward()
    
    optimizer_alp.step()
    optimizer_gcn.step()
    
    train_acc, val_acc, tmp_test_acc = test(gcn, data)
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        test_acc = tmp_test_acc
    log(Epoch=epoch, Loss=loss, Train=train_acc, Val=val_acc, Test=test_acc, Loss2 = loss2)
print(f'Best Val Acc: {best_val_acc:.4f}', f'Test Acc: {test_acc:.4f}')

Epoch: 001, Loss: 1.9654680490493774, Train: 0.5714, Val: 0.2300, Test: 0.2120, Loss2: 1.8164259195327759
Epoch: 002, Loss: 1.8978215456008911, Train: 0.5714, Val: 0.2620, Test: 0.2420, Loss2: 1.8163565397262573
Epoch: 003, Loss: 1.8485069274902344, Train: 0.7143, Val: 0.3220, Test: 0.3020, Loss2: 1.8164395093917847
Epoch: 004, Loss: 1.770589828491211, Train: 0.7143, Val: 0.3400, Test: 0.3360, Loss2: 1.816402792930603
Epoch: 005, Loss: 1.696747899055481, Train: 0.8571, Val: 0.3820, Test: 0.3910, Loss2: 1.8162449598312378
Epoch: 006, Loss: 1.5720155239105225, Train: 0.8571, Val: 0.4080, Test: 0.4070, Loss2: 1.8161628246307373
Epoch: 007, Loss: 1.5604878664016724, Train: 0.8571, Val: 0.3980, Test: 0.4070, Loss2: 1.8162505626678467
Epoch: 008, Loss: 1.520721673965454, Train: 0.8571, Val: 0.4040, Test: 0.4070, Loss2: 1.8163783550262451
Epoch: 009, Loss: 1.3960915803909302, Train: 0.8571, Val: 0.4280, Test: 0.4210, Loss2: 1.8164405822753906
Epoch: 010, Loss: 1.2945082187652588, Train: 0.857

In [ ]:
# for p in model.parameters():
#     print(p)

In [ ]:
optimizer = torch.optim.Adam(gcn.parameters(), lr=0.0005, weight_decay=5e-4)
best_val_acc = final_test_acc = 0
for epoch in range(1, 100):
    loss = train(gcn, data, optimizer, loss='cross_entropy')
    train_acc, val_acc, tmp_test_acc = test(gcn, data)
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        test_acc = tmp_test_acc
    log(Epoch=epoch, Loss=loss, Train=train_acc, Val=val_acc, Test=test_acc)
print(f'Best Val Acc: {best_val_acc:.4f}', f'Test Acc: {test_acc:.4f}')

In [ ]:
# outs.shape, outs[0, 0, :].sum(), (outs[0, 0, :] == outs[2, 0, :]).sum()

In [ ]:
best_val_acc = final_test_acc = 0

model.train()
for epoch in range(1, 100):
    optimizer.zero_grad()
    loss = model.loss()
    loss.backward()
    optimizer.step()
    
    train_acc, val_acc, tmp_test_acc = model.test()
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        test_acc = tmp_test_acc
    log(Epoch=epoch, Loss=loss, Train=train_acc, Val=val_acc, Test=test_acc)